# 项目模块化

cython无法自己作为程序入口,要么用python写程序入口,要么用C写程序入口,因此cython只能用于写模块,无论是在桥接python时还是在桥接C时都是如此.



在介绍项目模块化之前,先要分清楚是在什么环境下的模块化.cython的模块化按环境可以分为3种:

1. 作为cython模块的模块化
2. 作为python模块的模块化
3. 作为C模块的模块化

这三者有共同点也有区别,下面开始分别介绍这三种环境下的模块化异同.

本文的演示代码在[]()

## 作为cython模块的模块化

作为cython模块,它的主要作用是让cython代码间可以模块化编程,从而实现抽象和复用,是配合`cimport`语法的的代码组织工具.比如我们可以写一个cython模块A,当在一个新的cython项目中需要用到A中定义过的C函数或扩展类型中定义的C方法时我们就可以用`cimport`语法导入这个模块并使用了.因此cython模块可以看做是作为python模块的模块化和作为C模块的模块化的前提,后两者是前者的应用.只有弄明白如何作为cython模块进行模块化才能在后两种环境下进行的下去.

cython模块主要是看声明文件`.pxd`,和python的模块系统非常类似,一个cython项目如果要作为cython模块必须满足两种形式中的一种:

+ 简单模块,可以是一个单独的`.pxd`
+ 复杂模块,可以是一个带`__init__.pxd`的文件夹.

cython的复杂模块允许内部继续包含子模块,以我们的例子`mymath`为例:

```txt
mymath---\
    |---__init__.pxd
    |---mymath.pyx # 示例内部调用
    |---inner---\
    |       |---__init__.pxd
    |       |---l2norm.pxd   # 有接口`l2norm`
    |       |---l2norm.pyx
    |
    |---median_along_axis0.pxd  # 有接口`_median_along_axis0`
    |---median_along_axis0.pyx
    |---normalize.pxd  # 有接口`_normalize`
    |---normalize.pyx
    |---notexist.pxd #示例有声明无实现
    ...
callmymath.pyx # 示例外部调用
callmymath.py # 示例外部调用
```

### 内部调用

在其中的子模块可以和python中一样,使用相对引用导入需要的接口,如果是纯净模式则还是只能老老实实绝对引用导入,比如在一个新实现文件`mymath.pyx`中要导入接口`l2norm`和`_normalize`:

+ `mymath.pyx`(cython语法)

```cython
...
from .inner.l2norm cimport l2norm
from .normalize cimport _normalize
...
```

+ `mymath.pyx`纯净模式语法

```python
from cython.cimports.mymath.inner.l2norm import l2norm
from cython.cimports.mymath.normalize import _normalize
```

当然了Cython中同样要避免钻石引用.

### 外部调用

对于外部调用,无论哪种语法都只能老老实实绝对调用

+ `callmymath.pyx`(cython语法)

    ```cython
    from mymath.normalize cimport _normalize
    import numpy as np


    def callmymath():
        cdef double[:] output = _normalize(np.array([1.1,2.2,3.3,4.4]))
        print(np.asarray(output))
    ```

+ `callmymath.py`(纯净模式语法)

```cython
import cython
from cython.cimports.mymath.normalize cimport _normalize
import numpy as np


def callmymath():
    output: cython.double[:] = _normalize(np.array([1.1,2.2,3.3,4.4]))
    print(np.asarray(output))
```
### 没有实现的接口

cython的模块系统仅仅只是解决代码组织问题,无论哪种形式,光有声明文件也是没用的.使用`cimport`导入cython模块必须要有对应的实现部分,这个实现部分可以是模块中源文件`Cythonize`编译后的动态链接库,也可以是外部C/C++代码或库.如果没有,在运行时导入模块时一样会报`ModuleNotFoundError`错.

### Cython模块的编译

`Cythonize`编译出的是python可以调用的动态链接库模块,必须指定实现部分的源文件.无论是cython的简单模块还是复杂模块,编译后都是每个实现部分源文件都会编译出一个python可以调用的动态链接库文件.因此可以说cython模块的编译本质上是将其中的所有实现部分源文件分别编译为python可以调用的动态链接库文件.

`cythonize`支持指定的资源可以是单一的cython实现部分源码也可以是复杂模块,比如上面的例子,我们编译`callmymath.pyx`可以使用命令`cythonize -i --3 callmymath.pyx`,而编译复杂模块`mymath`则可以直接使用`cythonize -i --3 mymath`.

## 作为python模块的模块化

python本身就可以import由`cythonize`编译好的动态链接库每个动态链接库对应的模块名就是他们源文件的模块名.如果我们编译的出的是个单文件的模块,那么没什么好纠结的,直接import使用就好;而如果我们的项目本身是一个cython的复杂模块,由于`cythonize`编译复杂模块会将其中所有的`.py`,`.pyx`源文件都编译成各自的动态链接库,因此这里会有3种选择:

+ 不带`__init__.py`或`__init__.pyx`.python正常模块用`__init__.py`识别,没有它就会作为[命名空间包](https://blog.hszofficial.site/TutorialForPython/#/%E8%AF%AD%E6%B3%95%E7%AF%87/%E6%A8%A1%E5%9D%97/%E5%91%BD%E5%90%8D%E7%A9%BA%E9%97%B4%E5%8C%85)处理.这虽然不影响使用,但多少会影响加载速度.
+ 带`__init__.py`或`__init__.pyx`.python将可以作为正常模块使用,但会被`cythonize`编译.额外增加一个100多k的动态连接库.当然你也可以之后删掉,不过如果模块构造复杂这就太麻烦了.

因此为了实现我们的目标我们可以将复杂模块作为简单模块进行编译--我们在构造模块时让模块中只有`__init__.py`是`.py`文件,其他的源文件都是`.pyx`文件,至于声明文件`.pxd`则成了可选项,只要不影响cython模块的内部调用,有没有就无所谓了.

```
mymath---\
    |---__init__.pxd
    |---__init__.py # 让python可以识别为模块
    |---inner---\
    |       |---__init__.pxd
    |       |---__init__.py
    |       |---l2norm.pxd
    |       |---l2norm.pyx
    |
    |---median_along_axis0.pxd
    |---median_along_axis0.pyx
    |---normalize.pxd
    |---normalize.pyx
    ...
```

编译时指定source不再指定模块目录而是使用通配符`*`查找其中的所有`.pyx`文件进行编译,用上面的例子就是`cythonize -i --3 mymath/**/*.pyx`.

这样编译好后我们的模块就可以直接在python解释器中用`import`语法正常导入了.


## 作为C模块的模块化

C/C++中模块化使用的是头文件和`#include`语法,这意味着如果我们希望在C/C++中调用cython模块则需要让cython模块提供头文件.在Cython中有两种方式声明出来的接口可以提供头文件

+ `public`关键字
+ `api`关键字

这两个关键字都需要放在`cdef`后面.

### public关键字

public关键字仅可以用于修饰定义C中有的结构声明,也就是说不能用于扩展类型.

```cython
cdef public struct Bunny:  # struct声明
    int vorpalness

cdef public int spam  # 变量声明

cdef public void grail(Bunny *)  # 函数声明
```

`public`关键字会在转译步骤生成以一个和声明文件同名的头文件,且被声明为`public`的接口就会在这个文件中.在C/C++中需要使用这些接口时使用`#include`语法导入即可.


### api关键字


